# Polars' craziest* feature

*debatable

## With: Marco Gorelli (Quansight Labs, volunteer maintainer Polars)

In [1]:
import polars as pl

## Wait, what's Polars?

- DataFrame library
- Written in Rust, with Python bindings
- Blazingly fast!
- Syntax more like PySpark than pandas

## "I've heard it's super-fast"

![](duckdb_benchmark.png)

![](tpch.png)

## Right then, let's try it!

In [2]:
import numpy as np
df = pl.DataFrame({'x': np.random.randn(50_000_000)})
df.head()

x
f64
-0.122028
0.767915
-0.283176
-0.418662
-0.14506


In [3]:
%%time
result = df.with_columns(
    x_squared = pl.col.x.map_elements(lambda x: x**2),
    x_cubed = pl.col.x.map_elements(lambda x: x**3),
)
result.head()

<timed eval>:2: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - pl.col("x").map_elements(lambda x: ...)
  + pl.col("x") ** 2

<timed eval>:3: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - pl.col("x").map_elements(lambda x: ...)
  + pl.col("x") ** 3



CPU times: user 22 s, sys: 9.65 s, total: 31.6 s
Wall time: 32.7 s


x,x_squared,x_cubed
f64,f64,f64
-0.122028,0.014891,-0.001817
0.767915,0.589693,0.452834
-0.283176,0.080189,-0.022708
-0.418662,0.175278,-0.073382
-0.14506,0.021042,-0.003052
-1.456014,2.119978,-3.086718
-0.192288,0.036974,-0.00711
1.665041,2.772362,4.616097
1.413519,1.998036,2.824263


In [7]:
%%time
result = df.with_columns(
    x_squared = pl.col("x") ** 2,
    x_cubed = pl.col("x") ** 3,
)
result.head()

CPU times: user 202 ms, sys: 142 ms, total: 345 ms
Wall time: 221 ms


x,x_squared,x_cubed
f64,f64,f64
-0.122028,0.014891,-0.001817
0.767915,0.589693,0.452834
-0.283176,0.080189,-0.022708
-0.418662,0.175278,-0.073382
-0.14506,0.021042,-0.003052


We got a ????x performance increase for free!

## How does this work?

### 1. disassemble function into bytecode:

In [8]:
import dis

list(dis.get_instructions(lambda x: x**2))

[Instruction(opname='LOAD_FAST', opcode=124, arg=0, argval='x', argrepr='x', offset=0, starts_line=3, is_jump_target=False),
 Instruction(opname='LOAD_CONST', opcode=100, arg=1, argval=2, argrepr='2', offset=2, starts_line=None, is_jump_target=False),
 Instruction(opname='BINARY_POWER', opcode=19, arg=None, argval=None, argrepr='', offset=4, starts_line=None, is_jump_target=False),
 Instruction(opname='RETURN_VALUE', opcode=83, arg=None, argval=None, argrepr='', offset=6, starts_line=None, is_jump_target=False)]

### 2. parse the bytecode to figure out what the user wrote

`x`, `2`, `"binary power"` ==> `x**2`

### 3. educate the user on how they could have written their code more efficiently!

```diff
- pl.col("x").map_elements(lambda x: ...)
+ pl.col("x") ** 2
```

## But...why? Why not just do it the fast way for users?

A: Because then, users wouldn't learn!

Unfortunately, the warning above can only be emitted for relatively simple cases.

Educating users > doing things for them but only in some cases

## What more can Polars do for me?

Reach out to me on LinkedIn: https://www.linkedin.com/in/marcogorelli/

I post Polars tips once every whenever I feel like it

And also offer **Polars corporate training**